In [2]:
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

2024-02-08 13:33:28.133117: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-08 13:33:28.136885: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-08 13:33:28.207185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 13:33:28.207230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 13:33:28.208368: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
# Model Definition

# Modality Specific Module
class ModalitySpecificModule:
    @staticmethod
    def extract_modality_specific_interactions(input_layer):
        bi_gru_output = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(input_layer)
        msl_l1 = tf.keras.layers.Dense(128, activation='relu')(bi_gru_output)
        msl_output = tf.keras.layers.Dense(64, activation='relu')(msl_l1)
        return msl_output

    # Input Definition
    text_input = tf.keras.Input(shape=(50, 300))
    audio_input = tf.keras.Input(shape=(50, 5))
    visual_input = tf.keras.Input(shape=(50, 20))

    @staticmethod
    def compute(text_input, audio_input, visual_input):
        # Extract Modality Specific Interactions
        text_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(text_input)
        audio_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(audio_input)
        visual_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(visual_input)
        return text_msm_output, audio_msm_output, visual_msm_output


# Dense Multimodal Fusion Module
class DenseMultimodalFusionModule:
    residual_features = []
    @staticmethod
    def df_block(df_input1, df_input2, df_input3):
        df_output1 = tf.keras.layers.Dense(64, activation='relu')(df_input1)
        df_output2 = tf.keras.layers.Dense(64, activation='relu')(df_input2)
        df_output3 = tf.keras.layers.Dense(64, activation='relu')(df_input3)
        return df_output1, df_output2, df_output3
        
    @staticmethod
    def dense_fusion_layer(f1, f2, f3):
        df_input1 = tf.keras.layers.Concatenate()([f1, f3])
        df_input2 = tf.keras.layers.Concatenate()([f1, f2])
        df_input3 = tf.keras.layers.Concatenate()([f2, f3])
        r = tf.add(tf.add(f1, f2), f3)
        DenseMultimodalFusionModule.residual_features.append(r)
        fusion_output = DenseMultimodalFusionModule.df_block(df_input1, df_input2, df_input3)
        return fusion_output
    
    @staticmethod
    def compute(f1_0, f2_0, f3_0):
        f1_1, f2_1, f3_1 = DenseMultimodalFusionModule.dense_fusion_layer(f1_0, f2_0, f3_0)
        f1_2, f2_2, f3_2 = DenseMultimodalFusionModule.dense_fusion_layer(f1_1, f2_1, f3_1)
        f1_3, f2_3, f3_3 = DenseMultimodalFusionModule.dense_fusion_layer(f1_2, f2_2, f3_2)
        r = tf.add(tf.add(f1_3, f2_3), f3_3)
        DenseMultimodalFusionModule.residual_features.append(r)
        return DenseMultimodalFusionModule.residual_features


# Multimodal Residual Module
class MultimodalResidualModule:
    @staticmethod
    def compute(residual_features):
        final_residual_feature = residual_features[0]
        for r in residual_features[1:]:
            final_residual_feature = tf.add(final_residual_feature, r)
        return final_residual_feature


# Sentiment Classification Module
class SentimentClassificationModule:
    @staticmethod
    def compute(residual_feature):
        output_l1 = tf.keras.layers.Dense(64, activation='relu')(residual_feature)
        output_l2 = tf.keras.layers.Dense(32, activation='relu')(output_l1)
        output_l3 = tf.keras.layers.Dense(16, activation='relu')(output_l2)
        output_l4 = tf.keras.layers.Dense(8, activation='relu')(output_l3)

        # Modify the final layer to use tanh activation function
        output = tf.keras.layers.Dense(1, activation='tanh')(output_l4)
        # Scale and shift the output to be in the range [-3, 3]
        sentiment_output = 3 * output

        return sentiment_output

text_input, audio_input, visual_input = ModalitySpecificModule.text_input, ModalitySpecificModule.audio_input, ModalitySpecificModule.visual_input
text_msm_output, audio_msm_output, visual_msm_output = ModalitySpecificModule.compute(text_input, audio_input, visual_input)
residual_features = DenseMultimodalFusionModule.compute(text_msm_output, audio_msm_output, visual_msm_output)
final_residual_feature = MultimodalResidualModule.compute(residual_features)
output = SentimentClassificationModule.compute(final_residual_feature)


model = tf.keras.Model(inputs=[text_input, audio_input, visual_input], outputs=output)

model.summary()

2024-02-08 13:33:34.132510: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 13:33:34.236108: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 300)]            0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 50, 5)]              0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 50, 20)]             0         []                            
                                                                                                  
 bidirectional (Bidirection  (None, 128)                  140544    ['input_1[0][0]']             
 al)                                                                                          

In [4]:
model.compile(
  optimizer='adam', 
  loss=tf.keras.losses.MeanAbsoluteError(), 
  metrics=[
    tf.keras.metrics.Accuracy(),
    tf.keras.metrics.F1Score()
  ]
)

In [5]:
import pickle

with open('mosi_data.pkl', 'rb') as fp:
    data = pickle.load(fp)

In [6]:
print(data.keys())
print(data['train'].keys())
print(data['train']['vision'].shape)
print(data['train']['audio'].shape)
print(data['train']['text'].shape)
print(data['train']['labels'].shape)
print(data['train']['id'][0].shape)

dict_keys(['valid', 'test', 'train'])
dict_keys(['vision', 'labels', 'text', 'audio', 'id'])
(1284, 50, 20)
(1284, 50, 5)
(1284, 50, 300)
(1284, 1, 1)
(3,)


In [7]:
train_text_input = data['train']['text']
train_audio_input = data['train']['audio']
train_visual_input = data['train']['vision']
train_labels = data['train']['labels'][:, 0]

valid_text_input = data['valid']['text']
valid_audio_input = data['valid']['audio']
valid_visual_input = data['valid']['vision']
valid_labels = data['valid']['labels'][:, 0]

In [8]:
# Convert lists to numpy arrays
training_dataset = [train_text_input, train_audio_input, train_visual_input]
validation_data = ([valid_text_input, valid_audio_input, valid_visual_input], valid_labels)

In [10]:
model.fit(training_dataset, train_labels, validation_data=validation_data, epochs=10)

Epoch 1/10
41/41 [==============================] - 18s 168ms/step - loss: 1.2327 - accuracy: 0.0000e+00 - f1_score: 0.3799 - val_loss: 1.2097 - val_accuracy: 0.0000e+00 - val_f1_score: 0.4727
Epoch 2/10
41/41 [==============================] - 4s 88ms/step - loss: 1.0332 - accuracy: 0.0000e+00 - f1_score: 0.3799 - val_loss: 1.1188 - val_accuracy: 0.0000e+00 - val_f1_score: 0.4727
Epoch 3/10
41/41 [==============================] - 4s 87ms/step - loss: 0.8685 - accuracy: 0.0000e+00 - f1_score: 0.3799 - val_loss: 1.1305 - val_accuracy: 0.0000e+00 - val_f1_score: 0.4727
Epoch 4/10
41/41 [==============================] - 3s 85ms/step - loss: 0.8040 - accuracy: 0.0000e+00 - f1_score: 0.3799 - val_loss: 1.0524 - val_accuracy: 0.0000e+00 - val_f1_score: 0.4727
Epoch 5/10
41/41 [==============================] - 4s 86ms/step - loss: 0.7537 - accuracy: 0.0000e+00 - f1_score: 0.3799 - val_loss: 1.0288 - val_accuracy: 0.0000e+00 - val_f1_score: 0.4727
Epoch 6/10
41/41 [=========================